<a href="https://colab.research.google.com/github/Annie00000/Project/blob/main/5_13.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### dcc.store (沒有dcc.interval)

In [ ]:
import dash
from dash import html, dcc, Input, Output, State, ctx
import time

app = dash.Dash(__name__)
server = app.server

step_names = [
    "撈取資料",
    "確認與篩選資料",
    "資料預處理",
    "進行 Rule Check",
    "上拋資料"
]

# 預設狀態結構
default_progress = {
    "current_step": -1,
    "step_status": ["pending"] * len(step_names),  # 'pending' / 'running' / 'success' / 'error'
    "step_times": [None] * len(step_names),
    "done": False
}

app.layout = html.Div([
    html.H2("報告產出流程 - 每步 callback 管理"),
    dcc.Store(id='progress-store', data=default_progress),
    html.Button("開始產出報告", id="start-btn", n_clicks=0),
    html.Div(id="step-list", style={"marginTop": "20px", "fontSize": "16px"}),
    html.Div(id="final-button", style={"marginTop": "20px"})
])

# ===== Step Dispatcher ===== #
@app.callback(
    Output("progress-store", "data", allow_duplicate=True),
    Input("start-btn", "n_clicks"),
    prevent_initial_call=True
)
def start_process(n_clicks):
    # reset 狀態
    return default_progress.copy()

# ===== 每個步驟 Callback ===== #
@app.callback(
    Output("progress-store", "data", allow_duplicate=True),
    Input("progress-store", "data"),
    prevent_initial_call=True
)
def run_steps(data):
    for i in range(len(step_names)):
        if data["step_status"][i] == "pending":
            data["step_status"][i] = "running"
            data["current_step"] = i
            start = time.time()

            # 模擬處理
            time.sleep(1.0)

            # 模擬錯誤（可以移除）
            # if i == 2:
            #     data["step_status"][i] = "error"
            #     return data

            data["step_status"][i] = "success"
            data["step_times"][i] = round(time.time() - start, 2)
            break
    else:
        data["done"] = True

    return data

# ===== 顯示步驟清單 UI ===== #
@app.callback(
    Output("step-list", "children"),
    Output("final-button", "children"),
    Input("progress-store", "data")
)
def update_ui(data):
    current = data["current_step"]
    statuses = data["step_status"]
    times = data["step_times"]

    step_elements = []
    for i, step in enumerate(step_names):
        status = statuses[i]
        if status == "pending":
            color = "gray"
            symbol = "□"
        elif status == "running":
            color = "orange"
            symbol = "▶"
        elif status == "success":
            color = "green"
            symbol = "✔"
        elif status == "error":
            color = "red"
            symbol = "✖"
        else:
            color = "gray"
            symbol = "□"

        time_text = f"（{times[i]} 秒）" if times[i] else ""
        step_elements.append(html.Div(f"{symbol} {step} {time_text}", style={"color": color}))

    final_btn = html.Button("前往撰寫評論", id="to-comment") if data["done"] else ""

    return step_elements, final_btn

if __name__ == "__main__":
    app.run_server(debug=True)


### 補充 : 5/12 異常中斷 (有 func)

In [ ]:
import dash
from dash import html, dcc, Input, Output, State
import time
import threading
import pandas as pd
import numpy as np
import random

app = dash.Dash(__name__)
server = app.server

step_names = [
    "撈取資料",
    "確認與篩選資料",
    "資料預處理",
    "進行 Rule Check",
    "上拋資料"
]

# 初始進度資訊
progress = {
    'current_index': -1,
    'step_times': [None] * len(step_names),
    'step_status': ['pending'] * len(step_names),  # pending / success / error / running
    'error': False,
    'last_update_time': None
}

# 子程序：模擬流程
def run_report_process():
    for i in range(len(step_names)):
        if progress['error']:
            break

        progress['current_index'] = i
        progress['step_status'][i] = 'running'
        progress['last_update_time'] = time.time()

        time.sleep(1.5)  # 模擬執行時間

        # 模擬中斷的機率（可選）
        if i == 0 and random.random() < 0.05:
            # 模擬 crash：不設定 success，不更新 last_update_time
            return  # 中斷流程

        progress['step_times'][i] = round(time.time() - progress['last_update_time'], 2)
        progress['step_status'][i] = 'success'
        progress['last_update_time'] = time.time()

# Dash UI Layout
app.layout = html.Div([
    html.H2("報告產出流程"),
    html.Button("開始產出報告", id="run-btn", n_clicks=0),
    html.Div(id="step-list", style={"marginTop": "20px", "fontSize": "16px"}),
    html.Div(id="progress-label", style={"marginTop": "20px", "fontSize": "16px", "color": "red"}),
    html.Div(id="to-comment-btn", style={"marginTop": "20px"}),  # 動態產生按鈕
    dcc.Interval(id="interval", interval=1000, n_intervals=0, disabled=True),
])

# 開始按鈕：重置狀態並啟動執行緒
@app.callback(
    Output("interval", "disabled"),
    Input("run-btn", "n_clicks"),
    prevent_initial_call=True
)
def start_report(n_clicks):
    progress['current_index'] = -1
    progress['step_times'] = [None] * len(step_names)
    progress['step_status'] = ['pending'] * len(step_names)
    progress['error'] = False
    progress['last_update_time'] = time.time()

    thread = threading.Thread(target=run_report_process)
    thread.start()
    return False

# 更新 UI 每秒
@app.callback(
    Output("step-list", "children"),
    Output("progress-label", "children"),
    Output("interval", "disabled"),
    Output("to-comment-btn", "children"),
    Input("interval", "n_intervals")
)
def update_ui(n):
    current = progress['current_index']
    steps_total = len(step_names)
    step_status = progress['step_status']
    step_times = progress['step_times']
    last_update = progress['last_update_time']
    now = time.time()

    # 偵測 timeout（10 秒未更新）
    if last_update and (now - last_update > 10) and current < steps_total - 1:
        progress['error'] = True
        error_msg = f"⚠️ 流程中斷：{step_names[current]} 無回應超過 10 秒"
        return _build_step_list(), error_msg, True, ""

    # 流程完成：顯示按鈕
    if all(s in ['success'] for s in step_status):
        return _build_step_list(), "完成所有步驟！", True, html.Button("to_comment", id="to-comment")

    # 一般流程更新
    if progress['error']:
        return _build_step_list(), "流程中斷，請檢查錯誤", True, ""

    return _build_step_list(), f"目前進度：{current + 1}/{steps_total}", False, ""

# 建立步驟清單顯示元件
def _build_step_list():
    items = []
    for i, step in enumerate(step_names):
        status = progress['step_status'][i]
        time_spent = f" ({progress['step_times'][i]}秒)" if progress['step_times'][i] else ""

        if status == 'pending':
            prefix, color = "□", "gray"
        elif status == 'running':
            prefix, color = "▶", "orange"
        elif status == 'success':
            prefix, color = "✔", "green"
        elif status == 'error':
            prefix, color = "❌", "red"
        else:
            prefix, color = "□", "gray"

        items.append(html.Div(f"{prefix} {step}{time_spent}", style={"color": color, "marginBottom": "4px"}))

    return items

if __name__ == '__main__':
    app.run_server(debug=True)
